In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!pip3 install pyro-ppl

In [4]:
import os
path = os.path.abspath(os.path.join(os.getcwd(),".."))
import sys
sys.path.append(os.path.dirname(os.getcwd()))
from dynamics_predict.defaults import DYNAMICS_PARAMS, HYPER_PARAMS

env_name = 'inverteddoublependulum'
data_path = path+'/data/dynamics_data/'+env_name+'/dynamics.npy'
param_dim = len(DYNAMICS_PARAMS[env_name+'dynamics'])
print('parameter dimension: ', param_dim)

train_data = np.load(data_path, allow_pickle=True)
print('number of samples in data: ', len(train_data))
# split data
data_s, data_a, data_param, data_s_ = [], [], [], []
for d in train_data:
    [s,a,param], s_ = d
    data_s.append(s)
    data_a.append(a)
    data_param.append(param)
    data_s_.append(s_)

data_s = np.array(data_s)
data_a = np.array(data_a)
data_param = np.array(data_param)
data_s_ = np.array(data_s_)

print(data_s.shape, data_a.shape, data_param.shape, data_s_.shape)

parameter dimension:  5
number of samples in data:  3549
(3549, 11) (3549, 1) (3549, 5) (3549, 11)


In [5]:
x = np.concatenate((data_s,data_a), axis=-1)
theta = data_param
y = data_s_
print(x.shape, y.shape)

s_dim = data_s.shape[-1]
a_dim = data_a.shape[-1]
param_dim = data_param.shape[-1]
latent_dim = 2

(3549, 12) (3549, 11)


In [57]:
from torch.distributions import Normal
import torch.nn.functional as F
import torch.nn as nn
import os

device = 'cpu'

class DynamicsEncoder(nn.Module):
    """ Dynamics parameters encoding network: (params) -> (latent code) """
    def __init__(self, param_dim, latent_dim, hidden_dim=32, hidden_activation=F.relu, output_activation=F.tanh, num_hidden_layers=2, lr=1e-3, gamma=0.99):
        super(DynamicsEncoder, self).__init__()
        
        self.hidden_activation = hidden_activation
        self.output_activation = output_activation
        self._param_dim = param_dim
        self.latent_dim = latent_dim
        self.num_hidden_layers = num_hidden_layers

        self.input_layer =  nn.Linear(self._param_dim, hidden_dim)
        self.hidden_layers = [nn.Linear(hidden_dim, hidden_dim) for _ in range(num_hidden_layers)]
        self.hidden_layers = nn.ModuleList(self.hidden_layers)  # Have to wrap the list layers with nn.ModuleList to coorectly make those parameters tracked by nn.module! Otherwise those params will not be saved!
        self.output_layer =  nn.Linear(hidden_dim, latent_dim)

        self.optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        # self.scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=0.99)

    def forward(self, x):
        if not isinstance(x, torch.Tensor):
            x = torch.Tensor(x)
        x=self.hidden_activation(self.input_layer(x))
        for hl in self.hidden_layers:
            x=self.hidden_activation(hl(x))
        x=self.output_layer(x)
        if self.output_activation is not None:
            x=self.output_activation(x)
        return x

## a standard NN
# class EmbeddingDynamicsNetwork(nn.Module):
#     """ Common class for dyanmics prediction network with dynamics embedding as input: (s,a, alpha) -> s' """
#     def __init__(self, s_dim, a_dim, latent_dim, hidden_dim=32, hidden_activation=F.relu, output_activation=F.tanh, num_hidden_layers=2, lr=1e-3, gamma=0.99):
#         super(EmbeddingDynamicsNetwork, self).__init__()
        
#         self.hidden_activation = hidden_activation
#         self.output_activation = output_activation
#         self.latent_dim = latent_dim
#         self.num_hidden_layers = num_hidden_layers

#         self.input_layer =  nn.Linear(s_dim+a_dim+self.latent_dim, hidden_dim)
#         self.hidden_layers = [nn.Linear(hidden_dim, hidden_dim) for _ in range(num_hidden_layers)]
#         self.hidden_layers = nn.ModuleList(self.hidden_layers)  # Have to wrap the list layers with nn.ModuleList to coorectly make those parameters tracked by nn.module! Otherwise those params will not be saved!
#         self.output_layer =  nn.Linear(hidden_dim, s_dim)

#         self.optimizer = torch.optim.Adam(self.parameters(), lr=lr)
#         # self.scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=0.99)

#     def forward(self, x):
#         if not isinstance(x, torch.Tensor):
#             x = torch.Tensor(x)
#         x=self.hidden_activation(self.input_layer(x))
#         for hl in self.hidden_layers:
#             x=self.hidden_activation(hl(x))
#         x=self.output_layer(x)
#         if self.output_activation is not None:
#             x=self.output_activation(x)
#         return x

## a handwritten NN
class EmbeddingDynamicsNetwork(nn.Module):
    """ Common class for dyanmics prediction network with dynamics embedding as input: (s,a, alpha) -> s' """
    def __init__(self, s_dim, a_dim, latent_dim, hidden_dim=32, hidden_activation=F.relu, output_activation=F.tanh, num_hidden_layers=2, lr=1e-3, gamma=0.99):
        super(EmbeddingDynamicsNetwork, self).__init__()
        
        in_size = s_dim+a_dim+latent_dim
        out_size = s_dim

        self.weights1 =  nn.Parameter(torch.randn(in_size, hidden_dim))
        self.bias1 = nn.Parameter(torch.randn(hidden_dim))
        self.weights2 =  nn.Parameter(torch.randn(hidden_dim, out_size))
        self.bias2 = nn.Parameter(torch.randn(out_size))
        self.relu = nn.ReLU()

        self.optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        # self.scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=0.99)

    def forward(self, x):
        if not isinstance(x, torch.Tensor):
            x = torch.Tensor(x)
        x = self.relu(x @ self.weights1 + self.bias1)
        y = x @ self.weights2 + self.bias2
        return y

## just a linear layer
# class EmbeddingDynamicsNetwork(nn.Module):
#     """ Common class for dyanmics prediction network with dynamics embedding as input: (s,a, alpha) -> s' """
#     def __init__(self, s_dim, a_dim, latent_dim, hidden_dim=32, hidden_activation=F.relu, output_activation=F.tanh, num_hidden_layers=2, lr=1e-3, gamma=0.99):
#         super(EmbeddingDynamicsNetwork, self).__init__()
        
#         in_size = s_dim+a_dim+latent_dim
#         out_size = s_dim

#         self.weights =  nn.Parameter(torch.randn(in_size, out_size))
#         self.bias = nn.Parameter(torch.randn(out_size))

#         self.optimizer = torch.optim.Adam(self.parameters(), lr=lr)
#         # self.scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=0.99)

#     def forward(self, x):
#         if not isinstance(x, torch.Tensor):
#             x = torch.Tensor(x)
#         y = x @ self.weights + self.bias
#         return y

class DynamicsParamsOptimizer():
    """ 
    Dynamics parameters optimization model (gradient-based) based on a trained 
    forward dynamics prediction network: (s, a, learnable_params) -> s_ with real-world data. 
    """
    def __init__(self, s_dim, a_dim, param_dim, latent_dim, hidden_dim=32, hidden_activation=F.relu, output_activation=None, num_hidden_layers=2, lr=1e-2, gamma=0.99):
        self.dynamics_model = EmbeddingDynamicsNetwork(s_dim, a_dim, latent_dim, hidden_dim, hidden_activation, output_activation, num_hidden_layers, lr, gamma).to(device)
        self.dynamics_encoder = DynamicsEncoder(param_dim, latent_dim, hidden_dim, hidden_activation, output_activation, num_hidden_layers, lr, gamma).to(device)
        self.optimizer = torch.optim.Adam(list(self.dynamics_model.parameters()) + list(self.dynamics_encoder.parameters()), lr=lr)

        self.loss = nn.MSELoss()

    def forward(self, x, theta):
        """ s,a concat with param (learnable) -> s_ """

        alpha = self.dynamics_encoder(theta)
        y_  = self.dynamics_model(torch.cat((x, alpha), axis=-1))
        
        return y_

    def update(self, data, epoch=200, model_save_path=None):
        (x, theta, y) = data
        if not isinstance(x, torch.Tensor):
            x = torch.Tensor(x).to(device)
        if not isinstance(theta, torch.Tensor):
            theta = torch.Tensor(theta).to(device)        
        if not isinstance(y, torch.Tensor):
            y = torch.Tensor(y).to(device)

        for ep in range(epoch):
            y_ = self.forward(x, theta)
            self.optimizer.zero_grad()
            loss = self.loss(y_, y)
            loss.backward()
            self.optimizer.step()
            if ep%100==0:
                print('epoch: {}, loss: {}'.format(ep, loss.item()))
                torch.save(self.dynamics_model.state_dict(), model_save_path+'dynamics_model')
                torch.save(self.dynamics_encoder.state_dict(), model_save_path+'dynamics_encoder')

In [38]:
#stage 1, learning forward dynamics and dynamics encoder

opt = DynamicsParamsOptimizer(s_dim, a_dim, param_dim, latent_dim)
data = (x, theta, y)
model_save_path = './model/test/'
os.makedirs(model_save_path, exist_ok=True)
opt.update(data, epoch=20000, model_save_path=model_save_path)

epoch: 0, loss: 153.38121032714844
epoch: 100, loss: 2.048553705215454
epoch: 200, loss: 0.9408065676689148
epoch: 300, loss: 0.6003185510635376
epoch: 400, loss: 0.4563097059726715
epoch: 500, loss: 0.3664354979991913
epoch: 600, loss: 0.30238857865333557
epoch: 700, loss: 0.2539259195327759
epoch: 800, loss: 0.22015471756458282
epoch: 900, loss: 0.19902829825878143
epoch: 1000, loss: 0.18519233167171478
epoch: 1100, loss: 0.1749226599931717
epoch: 1200, loss: 0.16718044877052307
epoch: 1300, loss: 0.16075584292411804
epoch: 1400, loss: 0.15416094660758972
epoch: 1500, loss: 0.1488868147134781
epoch: 1600, loss: 0.14406649768352509
epoch: 1700, loss: 0.1400667428970337
epoch: 1800, loss: 0.13641862571239471
epoch: 1900, loss: 0.13294389843940735
epoch: 2000, loss: 0.12990282475948334
epoch: 2100, loss: 0.1271929293870926
epoch: 2200, loss: 0.12473718822002411
epoch: 2300, loss: 0.12244393676519394
epoch: 2400, loss: 0.12024980783462524
epoch: 2500, loss: 0.11814117431640625
epoch: 260

In [84]:
#stage 2, using BNN and SVI to fit alpha
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
import torch.nn as nn
from pyro.infer.autoguide import AutoNormal, AutoDiagonalNormal
from pyro.infer import SVI, Trace_ELBO, Predictive
from tqdm.auto import trange, tqdm

In [114]:
# load test data
test_data_path = path+'/data/dynamics_data/'+env_name+'/test_dynamics.npy'
test_data = np.load(test_data_path, allow_pickle=True)
print('number of samples in dest data: ', len(test_data))
idx=8  # index of sample to test: 0-10
test_s = np.array(test_data[idx]['sa'])[:, :-1]
test_a = np.array(test_data[idx]['sa'])[:, -1:]
test_param = np.array(test_data[idx]['params'])
test_s_ = np.array(test_data[idx]['s_'])
print(test_s.shape, test_a.shape, test_param.shape, test_s_.shape)

# load model
# updater = DynamicsParamsOptimizer(state_dim, action_dim, param_dim, latent_dim, switch_dim, model_save_path)
# updater.model.load_state_dict(torch.load(model_save_path+'model', map_location=device))

number of samples in dest data:  10
(4401, 11) (4401, 1) (5,) (4401, 11)


In [115]:
test_num_samples = 1000
test_x = torch.from_numpy(np.concatenate((test_s,test_a), axis=-1)).float()[:test_num_samples]
test_y = torch.from_numpy(test_s_).float()[:test_num_samples]
test_param = torch.from_numpy(test_param).float()

x_dim = test_x.shape[1]
y_dim = test_y.shape[1]
print(test_x.shape, test_y.shape, test_param.shape)

torch.Size([1000, 12]) torch.Size([1000, 11]) torch.Size([5])


In [116]:
import copy

## for a linear layer
# class EmbeddingFit(PyroModule):
#     def __init__(self, latent_dim, dynamics_model):
#         super().__init__()
#         self.alpha = PyroSample(dist.Normal(0., 1.).expand([latent_dim]).to_event(1))
#         self.weight = copy.deepcopy(dynamics_model.weights.cpu())
#         self.bias = copy.deepcopy(dynamics_model.bias.cpu())
#         self.sigma = pyro.sample("sigma", dist.Uniform(0., 1.).expand([1]).to_event(1))

#     def forward(self, x, output=None):
#         batch_size = x.shape[0]
#         input = torch.cat((x, self.alpha.repeat([batch_size, 1])), axis=-1)
#         mu = input @ self.weight + self.bias
#         with pyro.plate("instances", batch_size):
#             return pyro.sample("obs", dist.Normal(mu, self.sigma).to_event(1),
#                                obs=output)

## for the handwritten NN
class EmbeddingFit(PyroModule):
    def __init__(self, latent_dim, dynamics_model):
        super().__init__()
        self.alpha = PyroSample(dist.Normal(0., 1.).expand([latent_dim]).to_event(1))
        # self.weights1 = copy.deepcopy(dynamics_model.weights1.cpu())
        # self.bias1 = copy.deepcopy(dynamics_model.bias1.cpu())
        # self.weights2 = copy.deepcopy(dynamics_model.weights2.cpu())
        # self.bias2 = copy.deepcopy(dynamics_model.bias2.cpu())
        # self.bias2 = torch.randn(11, requires_grad=False)
        self.dynamics_model = dynamics_model

        # self.sigma = pyro.sample("sigma", dist.Uniform(0., 1.).expand([1]).to_event(1))
        self.sigma = pyro.sample("sigma", dist.LogNormal(0., 1.).expand([1]).to_event(1))
        self.relu = nn.ReLU()

    def forward(self, x, output=None):
        batch_size = x.shape[0]
        input = torch.cat((x, self.alpha.repeat([batch_size, 1])), axis=-1)
        # x = self.relu(input @ self.weights1 + self.bias1)
        # mu = x @ self.weights2 + self.bias2
        mu = self.dynamics_model(input)
        with pyro.plate("instances", batch_size):
            return pyro.sample("obs", dist.Normal(mu, 0.01).to_event(1),  # TODO whether 0.01 or self.sigma, self.sigma does not seem to be updated
                               obs=output)

pyro.clear_param_store() # this is important in notebook; elease memory!
# pyro.set_rng_seed(1)

dynamics_model = EmbeddingDynamicsNetwork(s_dim, a_dim, latent_dim, hidden_dim=32, hidden_activation=F.relu, output_activation=None, num_hidden_layers=2, lr=1e-2, gamma=0.99).to(device)
model_save_path = './model/test/'
dynamics_model.load_state_dict(torch.load(model_save_path+'dynamics_model', map_location=device))
dynamics_model.eval()
for name, param in dynamics_model.named_parameters():
    param.requires_grad = False  # this is critical! set not gradient for the trained model, otherwise will be updated in Pyro
    # if name == 'bias1':
    #     print(name, param)

model = EmbeddingFit(latent_dim, dynamics_model)

x = test_x
y = test_y

print(x.shape, y.shape, x.dtype)
print(test_x.shape, test_y.shape, test_x.dtype)

guide = AutoDiagonalNormal(model)  # posterior dist. before learning AutoDiagonalNormal
svi = SVI(model, guide, pyro.optim.Adam({"lr": 0.01}), Trace_ELBO())  # parameters to optimize are determined by guide()
for step in range(3000):
    loss = svi.step(x, y) / y.numel()  # data in step() are passed to both model() and guide()
    
    if step % 100 == 0:
        for name, value in pyro.get_param_store().items():
            # if name == 'bias2':
            print(name, pyro.param(name))
        # print(model.dynamics_model.bias1)  # remain unchanged as long as model params requires_grad = False
        print("step {} loss = {:0.4g}".format(step, loss))


torch.Size([1000, 12]) torch.Size([1000, 11]) torch.float32
torch.Size([1000, 12]) torch.Size([1000, 11]) torch.float32
AutoDiagonalNormal.loc Parameter containing:
tensor([ 0.0586, -0.1217], requires_grad=True)
AutoDiagonalNormal.scale tensor([0.0991, 0.1010], grad_fn=<SoftplusBackward0>)
step 0 loss = 7.245e+05
AutoDiagonalNormal.loc Parameter containing:
tensor([-0.7362, -0.7385], requires_grad=True)
AutoDiagonalNormal.scale tensor([0.0886, 0.0878], grad_fn=<SoftplusBackward0>)
step 100 loss = 2.129e+05
AutoDiagonalNormal.loc Parameter containing:
tensor([-1.1605, -0.8964], requires_grad=True)
AutoDiagonalNormal.scale tensor([0.0791, 0.0763], grad_fn=<SoftplusBackward0>)
step 200 loss = 5.515e+04
AutoDiagonalNormal.loc Parameter containing:
tensor([-1.3813, -0.9548], requires_grad=True)
AutoDiagonalNormal.scale tensor([0.0754, 0.0666], grad_fn=<SoftplusBackward0>)
step 300 loss = 3.794e+04
AutoDiagonalNormal.loc Parameter containing:
tensor([-1.5088, -0.9793], requires_grad=True)
Au

In [117]:
for name, value in pyro.get_param_store().items():
    if 'Auto' in name:
        print(name, pyro.param(name))


AutoDiagonalNormal.loc Parameter containing:
tensor([-2.6293, -1.5988], requires_grad=True)
AutoDiagonalNormal.scale tensor([0.0580, 0.0280], grad_fn=<SoftplusBackward0>)


In [118]:
## true value
dynamics_encoder = DynamicsEncoder(param_dim, latent_dim, hidden_dim=32, hidden_activation=F.relu, output_activation=None, num_hidden_layers=2, lr=1e-2, gamma=0.99).to(device)
model_save_path = './model/test/'
dynamics_encoder.load_state_dict(torch.load(model_save_path+'dynamics_encoder', map_location=device))

true_encoding = dynamics_encoder(test_param)
print(true_encoding)

tensor([-1.6684, -0.9392], grad_fn=<AddBackward0>)


In [138]:
predictive = Predictive(model, guide=guide, num_samples=500)
x_test = x_train[:10]
print(x_test.shape)
preds = predictive(x_test)

y_pred = preds['obs'].T.detach().numpy().mean(axis=1)
y_std = preds['obs'].T.detach().numpy().std(axis=1)

print(y_pred, y_std)

# fig, ax = plt.subplots(figsize=(10, 5))
# ax.plot(x, y, 'o', markersize=1)
# ax.plot(x_test, y_pred)
# ax.fill_between(x_test, y_pred - y_std, y_pred + y_std,
#                 alpha=0.5, color='#ffcd3c')

torch.Size([10, 23])
[[-1.4620235  -0.9292353  -2.3858504  ... -1.5019815  -1.8051643
  -2.8081913 ]
 [ 1.2608469   1.4969504   2.1020446  ...  0.26011953  1.8149033
   1.5758591 ]
 [ 0.79152566  0.92237675  1.3267851  ... -0.35175693  1.1348053
   1.0248308 ]
 [-1.4267539  -0.08124174 -0.70871776 ... -0.19416465 -1.2698132
  -1.2513993 ]
 [-0.23448806 -1.2992532  -0.70612466 ... -2.1950686  -1.2392956
   0.31301618]] [[1.4301293  1.3278846  1.8229095  ... 1.5404499  1.7940197  2.0547945 ]
 [1.4298494  1.1652166  1.8705648  ... 1.3980244  1.4551692  0.9440567 ]
 [1.0242914  0.63166404 1.1168156  ... 0.56255925 0.6149589  0.73265785]
 [0.57261705 0.900117   0.68755174 ... 0.5758168  0.8907142  0.9437713 ]
 [0.91220987 1.0359749  1.8830494  ... 1.6415218  0.8795121  0.97397715]]


In [ ]:
class Model(PyroModule):
    def __init__(self, h1=20, h2=20):
        super().__init__()
        self.fc1 = PyroModule[nn.Linear](x_dim, h1)
        self.fc1.weight = PyroSample(dist.Normal(0., 1.).expand([h1, x_dim]).to_event(2))
        self.fc1.bias = PyroSample(dist.Normal(0., 1.).expand([h1]).to_event(1))
        self.fc2 = PyroModule[nn.Linear](h1, h2)
        self.fc2.weight = PyroSample(dist.Normal(0., 1.).expand([h2, h1]).to_event(2))
        self.fc2.bias = PyroSample(dist.Normal(0., 1.).expand([h2]).to_event(1))
        self.fc3 = PyroModule[nn.Linear](h2, y_dim)
        self.fc3.weight = PyroSample(dist.Normal(0., 1.).expand([y_dim, h2]).to_event(2))
        self.fc3.bias = PyroSample(dist.Normal(0., 1.).expand([y_dim]).to_event(1))
        self.relu = nn.ReLU()

    def forward(self, x, y=None):
        batch_size = x.shape[0]
        # x = x.reshape(-1, 1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        mu = self.fc3(x).squeeze()
        sigma = pyro.sample("sigma", dist.Uniform(0., 1.).expand([y_dim]).to_event(1))  # the to_event(1) is necessary, you’ll need to call .to_event(1) to use scalar distributions like Normal as a joint diagonal distributions over multiple variables: see: https://forum.pyro.ai/t/simple-gmm-in-pyro/3047/3
        # print(mu.shape, sigma.shape, y.shape)

        # with pyro.plate("data", batch_size):
        #     obs = pyro.sample("obs", dist.Normal(mu, sigma).to_event(1), obs=y) # the to_event(1) is necessary
        #     return mu

        with pyro.plate("instances", batch_size):
            return pyro.sample("obs", dist.Normal(mu, sigma).to_event(1),
                               obs=y)